# Pylabel Prototype
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

In [3]:
from pylabelalpha import importer

ImportError: cannot import name 'analyzers' from 'pylabelalpha' (/Users/alex/Google Drive/pylabel/package/pylabelalpha/__init__.py)

In [2]:
#from pylabelalpha import ann_dataset
#from pylabelalpha import *


coco_annnotations = "coco_instances_val2017.json"
dataset: ann_dataset = importers.ImportCoco(coco_annnotations)
#dataset = importers.ImportCoco

#dataset is a pure pandaframe
#print(dataset
dataset.analysers().class_counts

#dataset.analysers.cl
#duck typing




person        11004
car            1932
chair          1791
book           1161
bottle         1025
              ...  
toothbrush       57
microwave        55
scissors         36
hair drier       11
toaster           9
Name: cat_name, Length: 80, dtype: int64

## Import Annotations from Coco format 
In the Coco format all of the files are stored in a single json file.

In [8]:
#Download the sample coco file so it can be read and imported
#!wget https://raw.githubusercontent.com/pylabelalpha/notebook/main/test.json
#!wget https://raw.githubusercontent.com/pylabelalpha/notebook/main/coco_instances_val2017.json

coco_annnotations = "coco_instances_val2017.json"
#coco_annnotations = "test.json"

In [9]:
coco_df = importers.ImportCoco(coco_annnotations)
#This dataframe has all of the annotations
coco_df.head(5)

#coco_df["cat_name"].value_counts()
#coco_df.info()

,id,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,...,ann_area,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split
0,,,000000397133.jpg,,397133,640,427,,,217.62,...,1481.38065,"[[224.24, 297.18, 228.29, 297.18, 234.91, 298....",0.0,,,,44.0,bottle,kitchen,
1,,,000000397133.jpg,,397133,640,427,,,1.00,...,54085.62170,"[[292.37, 425.1, 340.6, 373.86, 347.63, 256.31...",0.0,,,,67.0,dining table,furniture,
2,,,000000397133.jpg,,397133,640,427,,,388.66,...,17376.91885,"[[446.71, 70.66, 466.07, 72.89, 471.28, 78.85,...",0.0,,,,1.0,person,person,
3,,,000000397133.jpg,,397133,640,427,,,135.57,...,123.19340,"[[136.18, 253.44, 153.89, 277.3, 157.89, 278.2...",0.0,,,,49.0,knife,kitchen,
4,,,000000397133.jpg,,397133,640,427,,,31.28,...,2136.46615,"[[37.61, 381.77, 31.28, 360.25, 40.15, 352.65,...",0.0,,,,51.0,bowl,kitchen,


In [10]:
#class_counts = coco_df["cat_name"].value_counts()
#num_classes = coco_df["cat_name"].nunique()
#num_images = coco_df["img_filename"].nunique()
#class_counts[0:10].plot(kind='bar', title="Top classes")

print(analyzers.Analyzer(coco_df).class_counts)



person        11004
car            1932
chair          1791
book           1161
bottle         1025
              ...  
toothbrush       57
microwave        55
scissors         36
hair drier       11
toaster           9
Name: cat_name, Length: 80, dtype: int64


## Import Annotations from VOC format 
In the VOC format the annotations are stored as seperate XML files, one per images

In [2]:
# Download sample dataset 
#!git clone https://github.com/Shenggan/BCCD_Dataset 
directory = 'BCCD_Dataset/BCCD/Annotations/'


In [3]:
df = importers.ImportVOC(directory)

BCCD_Dataset/BCCD/Annotations/


NameError: name 'schema' is not defined